In [ ]:
# !pip install azure-storage-blob>=12.19.0

In [ ]:
# !pip install docling

In [ ]:
# !pip install pytesseract

In [1]:
import os
import io
import imagehash
from PIL import Image

import uuid
import pytesseract
import secrets
import string

from pathlib import Path
from dotenv import load_dotenv

from docling.datamodel.base_models import InputFormat
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling_core.types.doc import PictureItem, TextItem

from azure.storage.blob import BlobServiceClient
from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions

from IPython.display import display, Markdown

In [2]:
load_dotenv()

True

In [3]:
connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")

openai_key = os.getenv("OPENAI_API_KEY")
if not openai_key:
    raise ValueError("OPENAI_API_KEY not found in environment")

In [4]:
from datetime import datetime, timedelta

def upload_and_get_sas(data, blob_path):
    # 1. Get blob client and upload
    service_client = BlobServiceClient.from_connection_string(connection_string)
    blob_client = service_client.get_blob_client(container='dummy', blob=blob_path)
    blob_client.upload_blob(data, overwrite=True)

    # 2. Generate SAS token valid for (e.g.) 1 year
    sas_token = generate_blob_sas(
        account_name=service_client.account_name,
        container_name='dummy',
        blob_name=blob_path,
        account_key=service_client.credential.account_key,
        permission=BlobSasPermissions(read=True),
        expiry=datetime.utcnow() + timedelta(days=365)
    )

    # 3. Return the full URL
    return f"{blob_client.url}?{sas_token}"

In [5]:
pipeline_options = PdfPipelineOptions()
pipeline_options.generate_picture_images = True
pipeline_options.images_scale = 2.0
pipeline_options.generate_page_images=True
pipeline_options.do_ocr=True
pipeline_options.do_table_structure=True
pipeline_options.generate_parsed_pages=True

In [6]:
# 2. Pass these options to the converter
converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

In [7]:
pdf_path = r"C:\Users\Harish\Workspace\maestro_projects\1440_Bot\source_docs\1440-Microsoft Multifactor Authentication Documentation.pdf"

In [ ]:
# 3. Proceed with your conversion and iteration
result = converter.convert(pdf_path)

2026-01-14 16:21:26,877 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-14 16:21:26,911 - INFO - Going to convert document batch...
2026-01-14 16:21:26,912 - INFO - Initializing pipeline for StandardPdfPipeline with options hash c00cd68e79f4403162cfad1c1430e139
2026-01-14 16:21:26,961 - INFO - Loading plugin 'docling_defaults'
2026-01-14 16:21:26,963 - INFO - Registered picture descriptions: ['vlm', 'api']
2026-01-14 16:21:27,058 - INFO - Loading plugin 'docling_defaults'
2026-01-14 16:21:27,062 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2026-01-14 16:21:27,504 - INFO - rapidocr cannot be used because onnxruntime is not installed.
2026-01-14 16:21:27,505 - INFO - easyocr cannot be used because it is not installed.
2026-01-14 16:21:27,788 - INFO - Accelerator device: 'cpu'
[INFO] 2026-01-14 16:21:27,809 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-14 16:21:27,817 [RapidOCR] device_config.py:50: Usi

In [ ]:
doc = result.document

In [ ]:
print(doc.export_to_markdown())

In [ ]:
IMAGE_DIR = "C:/Users/Harish/Workspace/maestro_projects/1440_Bot/hash_images"

def build_reference_hashes(ref_dir):
    ref_hashes = []
    # Supported formats
    valid_exts = (".png", ".jpg", ".jpeg")
    
    for filename in os.listdir(ref_dir):
        if filename.lower().endswith(valid_exts):
            path = os.path.join(ref_dir, filename)
            with Image.open(path) as img:
                # We use dhash here as it's more robust for icons/symbols
                h = imagehash.phash(img.convert("RGB"))
                ref_hashes.append(h)
                print(f"Added to Banned List: {filename} (Hash: {h})")
    return ref_hashes

BANNED_REF_LIST = build_reference_hashes(IMAGE_DIR)

In [ ]:
collected = []
seen_hashes = set()
PHASH_THRESHOLD = 15


# Iterate items in the exact order they appear in the PDF
for idx, (element, _level) in enumerate(doc.iterate_items()):
    if isinstance(element, TextItem):
        text = (element.text or "").strip()
        if text:
            collected.append({"type": "text", "content": text})
    
    elif isinstance(element, PictureItem):
        if element.image and element.image.pil_image:
            pil_img = element.image.pil_image
            
            # 2. Generate hash for filtering
            try:
                curr_hash = imagehash.phash(pil_img)
            except Exception as e:
                print(f"Error hashing image at index {idx}: {e}")
                continue
            
            # 3. Filter against BANNED_REF_LIST (Logos/Arrows/Icons)
            is_banned = any((curr_hash - ref) <= PHASH_THRESHOLD for ref in BANNED_REF_LIST)
            
            if is_banned:
                print(f"Skipping Image {idx}: Matched Banned Reference.")
                display(pil_img)
                continue
            
            # # 4. Filter against duplicates within the same document
            # is_duplicate = any((curr_hash - h) < PHASH_THRESHOLD for h in seen_hashes)
            # if is_duplicate:
            #     print(f"Skipping Image {idx}: Duplicate found in document.")
            #     display(pil_img)
            #     continue
            
            # 5. Add to seen hashes and collect the clean image
            seen_hashes.add(curr_hash)
            
            print(f"Keeping Image {idx}: Valid technical asset")
            display(pil_img)
            
            collected.append({
                "type": "image",
                "image": pil_img, 
                "index": idx,
                "page": element.prov[0].page_no if element.prov else 0,
                "bbox": element.prov[0].bbox if element.prov else None
            })

In [ ]:
# Containers for both versions
ocr_structural_markdown = [] # Version A: OCR Placeholders for reasoning
final_sas_markdown = []      # Version B: The SAS URL version you want to keep
asset_manifest_data = []      # The "Bridge" mapping table
vision_sas_urls = []         # For the model's visual input

project_name = "dummy"
storage_base_path = f"{project_name}/processed_images"

for item in collected:
    if item["type"] == "text":
        content = item["content"]
        ocr_structural_markdown.append(f"[DOC_TEXT]: {content}")
        final_sas_markdown.append(content)
    
    elif item["type"] == "image":
        pil_img = item["image"]
        page = item.get("page", 0)
        
        # --- PHASE 1: Generate the Shared ID ---
        # This ID ties the OCR text to the SAS URL permanently
        unique_hex = ''.join(secrets.choice(string.hexdigits.lower()) for _ in range(4))
        asset_id = f"ASSET_{unique_hex}"
        
        # --- PHASE 2: OCR Extraction ---
        try:
            ocr_text = pytesseract.image_to_string(pil_img).strip().replace("\n", " ")
        except Exception:
            ocr_text = "[No readable text found]"

        # --- PHASE 3: Original Storage Logic ---
        img_byte_arr = io.BytesIO()
        pil_img.save(img_byte_arr, format='PNG')
        img_bytes = img_byte_arr.getvalue()
        
        filename = f"visual_{unique_hex}_page_{page}.png"
        blob_path = f"{storage_base_path}/{filename}"
        
        # Keep your existing upload function exactly as it is
        sas_url = upload_and_get_sas(img_bytes, blob_path)

        # --- PHASE 4: Populate Both Markdown Versions ---
        
        # A. The OCR Version (Placeholders for LLM reasoning)
        img_placeholder = (
            f"\n--- IMAGE OCR PLACEHOLDER ---\n"
            f"SOURCE_ID: {asset_id}\n"
            f"TEXT_FOUND_IN_IMAGE: {ocr_text}\n"
            f"--- END OCR PLACEHOLDER ---\n"
        )
        ocr_structural_markdown.append(img_placeholder)
        
        # B. The SAS URL Version (Interleaved images with clickable links)
        # Using the unique_hex in alt-text for cross-referencing
        final_sas_markdown.append(f"![Document Visual {unique_hex}]({sas_url})")

        # C. The Manifest (The Source of Truth table)
        asset_manifest_data.append({
            "id": asset_id,
            "ocr": ocr_text,
            "url": sas_url
        })
        vision_sas_urls.append(sas_url)

# Final Strings
llm_reasoning_draft = "\n".join(ocr_structural_markdown)
llm_ready_sas_markdown = "\n\n".join(final_sas_markdown)

In [ ]:
# print(llm_reasoning_draft)

In [ ]:
# print(llm_ready_sas_markdown)

In [ ]:
# asset_manifest_data

In [ ]:
# vision_sas_urls

In [ ]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
client

In [ ]:
user_query = f"Explain multifactor authentication steps?"

In [ ]:
full_system_prompt = (
    """# System Prompt: Multi-Modal Technical Documentation Assistant

You are a technical documentation assistant that answers user queries by analyzing interleaved text, OCR data, and visual content from technical documents.

## YOUR INPUT STRUCTURE

You will receive four components:

1. **USER QUERY**: The specific question to answer
2. **IMAGE-TO-OCR MAPPING MANIFEST**: A lookup dictionary linking Asset IDs to OCR content and SAS URLs
3. **SOURCE DOCUMENT DRAFT**: Text content with OCR placeholders
4. **ACTUAL IMAGES**: The visual content to verify and understand context

## YOUR CORE TASK

Answer the user's query by:
1. **Semantic matching** between text instructions and visual content using OCR hints
2. **Logical reconstruction** of steps in proper sequence (Step 1 → 2 → 3...)
3. **Visual verification** using the actual images to confirm context
4. **Precise image binding** to ensure each step shows the correct screenshots

## CRITICAL IMAGE HANDLING RULES

### URL Fidelity (NON-NEGOTIABLE)
- Use `FINAL_SAS_URL` from the manifest EXACTLY as provided
- **NEVER modify, truncate, or regenerate these URLs**
- Include the complete signature: `?se=...&sp=...&sig=...`
- Format: `![Description](full_URL_with_all_parameters)`

### Semantic Matching Process
1. Read the step description from [DOC_TEXT] (e.g., "Scan QR code")
2. Check the manifest's `OCR_CONTENT` for matching keywords
3. Look at the actual image to verify it matches the description
4. Bind the correct `FINAL_SAS_URL` to that step

### Multi-Image Steps
- If multiple images relate to one step, include all relevant URLs sequentially
- Example: Step 4 might show 3 app screens - include all 3 with their exact URLs

## RESPONSE STRUCTURE

### For Procedural Queries (e.g., "Explain MFA steps")

Provide a brief overview, then break down each step:

**Step 1: [Action Title]**
Clear description of what the user should do.

![Description of what this shows](https://exact_sas_url_from_manifest...)

**Step 2: [Action Title]**
Clear description of the next action.

![Description of what this shows](https://exact_sas_url_from_manifest...)

### For FAQ Queries
Quote the relevant FAQ section and include any associated images with their exact URLs.

### For Specific Questions
Provide a direct answer using only information from the source material, with relevant images.

## LOGICAL RECONSTRUCTION RULES

The source draft may have steps out of order due to multi-column PDF parsing. You must:

1. **Identify all numbered steps** (Step One, Step Two, Step Three, etc.)
2. **Reorder them numerically** regardless of their position in the draft
3. **Match images semantically** using OCR content, not just position in document
4. **Verify with your vision** by examining what the images actually show.
5. **Order** If a step refers to an action shown in an image (e.g., 'Click the Accept button'), ensure the image containing that specific button is the one placed under that step.

**Example**: If the draft shows "Step Four" before "Step Three", but Step Three discusses permissions and you see an image with "Permissions requested" in it, that image belongs to Step Three.

## CONTENT QUALITY RULES

- **Remove parsing artifacts**: Strip `[DOC_TEXT]:`, `--- IMAGE OCR PLACEHOLDER ---`, `SOURCE_ID:`, etc.
- **Deduplicate**: Remove repeated "FAQ's" headers or redundant sections
- **Clean formatting**: Use proper markdown with bold for step titles
- **Be concise**: Answer the query directly, don't reconstruct the entire document unless asked

## BOUNDARIES & CONSTRAINTS

- **Use only provided information** - no external knowledge about products/services
- **If information is missing**: State "This detail is not available in the documentation"
- **For unanswerable queries**: Provide the fallback contact from the document
- **Trust what you see**: If OCR hints and actual image content conflict, trust the image
- **Missing images**: If a step mentions an image but you can't find it in the manifest, write "Visual not available"

## OUTPUT REQUIREMENTS

- Answer the user's query directly and professionally
- Include relevant images with their EXACT URLs from the manifest
- Use clear, structured formatting
- No meta-commentary about your reasoning process
- No explanations of how you processed the data
- Focus on being helpful and accurate"""
)

In [ ]:
reconstruction_rules = (
    """"1. ANALYSIS: Compare the instruction steps in [DOC_TEXT] against the [OCR_CONTENT] in the Manifest.
    2. ANCHORING: If 'Step Three' mentions 'Permissions' and an image OCR contains 'Permissions requested
    3."move that image URL to Step Three, regardless of where it appeared in the draft.
    4. LOGICAL ORDERING: Ensure the final output follows a strict numerical sequence (Step 1, 2, 3, ....).
    5. IMAGE PLACEMENT: Place the image(s) IMMEDIATELY after the step description.
    6. FIDELITY: Maintain the SAS URLs exactly as provided in the manifest. Do not truncate or modify them
    7. DEDUPLICATION: Remove any redundant Step text or duplicate FAQ sections found in the draft."""
)

In [ ]:
# Build the content array with text and images
content = []

# 1. User query
content.append({
    "type": "input_text",
    "text": f"### USER QUERY\n{user_query}\n\n"
})

# 2. Image manifest
content.append({
    "type": "input_text",
    "text": f"### IMAGE-TO-OCR MAPPING MANIFEST\n{asset_manifest_data}\n\n" 
})

# 3. Source document draft
content.append({
    "type": "input_text",
    "text": f"### SOURCE DOCUMENT DRAFT\n{llm_reasoning_draft}\n" 
})

# 4. Add all images
for image_url in vision_sas_urls:
    content.append({
        "type": "input_image",
        "image_url": image_url
    })

# # 5. Reconstruction rules (optional, could be in system prompt)
# content.append({
#     "type": "text",
#     "text": reconstruction_rules_text
# })

In [ ]:
response = client.responses.create(
    
    model="gpt-5.2",  # required in practice
    instructions=full_system_prompt,
    input=[
        {
            "role": "user",
            "content": content
        }
    ],
    
    reasoning={
        "effort": "high"
    }
)

In [ ]:
answer = response.output_text

In [ ]:
display(Markdown(answer))

In [ ]:
# full_system_prompt = (
#     """# System Prompt: Technical Documentation Assistant
# You are a technical documentation assistant that helps users understand procedures and information from provided documentation.

# ## Core Responsibility
# Answer user queries using ONLY the information in the provided markdown document. The document contains text instructions and embedded image SAS URLs that illustrate those instructions.

# ## Critical Rules for Image Handling

# ### Image URL Preservation
# - The markdown contains image SAS URLs in this format: `![Description](https://url...?signature...)`
# - **NEVER modify, truncate, or alter these URLs in any way**
# - Copy them EXACTLY as provided, including all query parameters and signatures
# - These URLs are time-sensitive and any modification will break them

# ### Image-Text Association
# - When explaining a step or concept, include the relevant image SAS URL(s) that illustrate it
# - Place the image or images reference immediately after describing what it shows
# - If multiple images relate to one explanation, include all relevant images in sequence
# - Format example:
#   ```
#   **Step 2: Enter Credentials**
#   Provide your username and password on the login screen.
  
#   ![Visual showing login screen](https://full_url_exactly_as_provided...)
#   ```

# ## Response Structure

# 1. **Direct answer first** - Start with a brief response to the user's question
# 2. **Sequential explanation** - Break down procedures step-by-step in logical order
# 3. **Visual references** - Include relevant images inline with explanations

# ## Content Boundaries

# - **Only use information from the provided documentation** - Never add external knowledge
# - **If information is missing or unclear**, state: "This information is not available in the provided documentation"
# - **If contact information exists in the document**, provide it when you cannot answer a query
# - **Never invent steps, details, or procedures** not explicitly in the source material

# ## Tone

# - Professional and helpful
# - Clear and concise
# - Action-oriented for procedural content
# - Factual without speculation"""
# )

In [ ]:
# # 1. Format the asset_manifest_data into a clear lookup table for the LLM
# manifest_reference_table = "### IMAGE-TO-OCR MAPPING MANIFEST\n"
# manifest_reference_table += "Use this table to match the textual instructions to the correct Image Asset ID and URL based on the OCR content.\n\n"

# for asset in asset_manifest_data:
#     manifest_reference_table += (
#         f"- ID: {asset['id']}\n"
#         f"  OCR_CONTENT: \"{asset['ocr']}\"\n"
#         f"  FINAL_SAS_URL: {asset['url']}\n"
#     )

# api_content = []

# # 1. Lead with the User's Query
# api_content.append({
#     "type": "input_text", 
#     "text": f"### USER QUERY / INTENT\n{user_query}\n\n"
# })

# # 2. Provide the Source of Truth (The Manifest)
# # This is the "dictionary" that ties OCR meaning to specific SAS URLs
# api_content.append({
#     "type": "input_text", 
#     "text": manifest_reference_table
# })

# # 3. Provide the Reasoning Draft (The interleaved version with placeholders)
# # This gives the model the narrative flow it needs to fix
# api_content.append({
#     "type": "input_text", 
#     "text": f"### SOURCE DOCUMENT DRAFT (REASONING VERSION):\n{llm_reasoning_draft}"
# })

# # 4. Provide the High-Res Vision Inputs
# # This allows the model to visually verify the OCR and see UI elements
# for url in vision_sas_urls:
#     api_content.append({
#         "type": "input_image",
#         "image_url": url
#     })

# # 5. Reasoning & Output Instructions
# api_content.append({
#     "type": "input_text",
#     "text": (
#         "\n--- RECONSTRUCTION RULES ---\n"
#         "1. ANALYSIS: Compare the instruction steps in [DOC_TEXT] against the [OCR_CONTENT] in the Manifest.\n"
#         "2. ANCHORING: If 'Step Three' mentions 'Permissions' and an image OCR contains 'Permissions requested', "
#         "move that image URL to Step Three, regardless of where it appeared in the draft.\n"
#         "3. LOGICAL ORDERING: Ensure the final output follows a strict numerical sequence (Step 1, 2, 3, 4, 5, 6).\n"
#         "4. IMAGE PLACEMENT: Place the image(s) IMMEDIATELY after the step description.\n"
#         "5. FIDELITY: Maintain the SAS URLs exactly as provided in the manifest. Do not truncate or modify them.\n"
#         "6. DEDUPLICATION: Remove any redundant Step text or duplicate FAQ sections found in the draft."
#     )
# })

In [ ]:
# api_content = []

# # 1. Lead with the User's Query
# # This ensures the model knows the specific intent (e.g., "Explain the steps")
# api_content.append({
#     "type": "input_text", 
#     "text": f"### USER QUERY / INTENT\n{user_query}\n\n"
# })

# # 2. Provide the Raw Interleaved Markdown
# # This is the primary data source containing text and the SAS URLs
# api_content.append({
#     "type": "input_text", 
#     "text": f"### SOURCE DOCUMENT (RAW MARKDOWN)\n{llm_ready_markdown}"
# })

# for url in vision_sas_urls:
#     api_content.append({
#         "type": "input_image",
#         "image_url": url
#     })

# # 4. Final Instruction Reinforcement
# # 4. Output constraints (not reasoning)
# api_content.append({
#     "type": "input_text",
#     "text": (
#         "OUTPUT INSTRUCTIONS:\n"
#         "- Use both text and images to correctly order the steps.\n"
#         "- Preserve all SAS URLs exactly.\n"
#         "- Attach images to the correct step.\n"
#         "- Do not guess if an image is unclear.\n"
#         "- You must understand and reason the visual content in the image and try to interleave that images/images under respective step.\n"
#         "- Use the textual explanation as logical step to interleave repective image/images.\n"
#     )
# })